In [1]:
import _pickle as cPickle
with open('data.pkl', 'rb') as fo:
        dict = cPickle.load(fo)

In [9]:
test = pd.read_csv('test.csv')

In [2]:
import pandas as pd
df = pd.DataFrame(dict)

In [4]:
data = df[[
    'date_block_num',
    'shop_id',
    'item_id',
    'item_cnt_month',
    'city_code',
    'item_category_id',
    'type_code',
    'subtype_code',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    'item_cnt_month_lag_6',
    'item_cnt_month_lag_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_2',
    'date_item_avg_item_cnt_lag_3',
    'date_item_avg_item_cnt_lag_6',
    'date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1',
    'date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_3',
    'date_shop_avg_item_cnt_lag_6',
    'date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    #'date_shop_type_avg_item_cnt_lag_1',
    #'date_shop_subtype_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    #'date_type_avg_item_cnt_lag_1',
    #'date_subtype_avg_item_cnt_lag_1',
    'delta_price_lag',
    'month',
    'days',
    'item_shop_last_sale',
    'item_last_sale',
    'item_shop_first_sale',
    'item_first_sale',
]]

In [5]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

Linear Regression

In [6]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
model_lr_history = model_lr.fit(X_train, Y_train)

In [11]:
Y_test = model_lr.predict(X_test)

submission = pd.DataFrame({
    "ID": test.index,
    "item_cnt_month": Y_test.flatten()
})
submission.to_csv('lr_submission.csv',index=False)

In [12]:
lr_pred = model_lr.predict(X_valid)

In [14]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(Y_valid, Y_pred))
rmse

0.9635806754524067

LASSO

In [16]:
from sklearn.linear_model import Lasso

model_lasso = Lasso(alpha=0.05)
model_lasso_history = model_lasso.fit(X_train, Y_train)

In [17]:
Y_pred = model_lasso.predict(X_valid)
rmse = sqrt(mean_squared_error(Y_valid, Y_pred))
rmse

1.0420949754721773

In [18]:
from sklearn.linear_model import Ridge
model_ridge = Ridge(alpha=0.01)
model_ridge.fit(X_train,Y_train) 
Y_pred = model_ridge.predict(X_valid)
rmse = sqrt(mean_squared_error(Y_valid, Y_pred))
rmse

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=7.99445e-11): result may not be accurate.
  overwrite_a=True).T


0.9635371886828042

In [19]:
from sklearn.linear_model import ElasticNet
model_enet = ElasticNet(alpha = 0.01)
model_enet.fit(X_train,Y_train) 
Y_pred = model_enet.predict(X_valid)
rmse = sqrt(mean_squared_error(Y_valid, Y_pred))
rmse

0.966454154255877

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# model_rf = RandomForestRegressor()
# model_rf.fit(X_train,Y_train) 
# Y_pred = model_rf.predict(X_valid)
# rmse = sqrt(mean_squared_error(Y_valid, Y_pred))
# rmse

Gradient Boost

In [23]:
from sklearn.ensemble import GradientBoostingRegressor
params = {'n_estimators':3, 'max_depth':3, 'learning_rate':1, 'criterion': 'mse'}

model_gb = GradientBoostingRegressor(**params)
model_gb.fit(X_train,Y_train)
Y_pred = model_gb.predict(X_valid)
rmse = sqrt(mean_squared_error(Y_valid, Y_pred))
rmse

0.9614517306710679

In [36]:
import pickle
pickle.dump(gb_pred, open('gb_train.pickle', 'wb'))
pickle.dump(gb_test, open('gb_test.pickle', 'wb'))

pickle.dump(enet_pred, open('enet_train.pickle', 'wb'))
pickle.dump(enet_test, open('enet_test.pickle', 'wb'))

pickle.dump(ridge_pred, open('ridge_train.pickle', 'wb'))
pickle.dump(ridge_test, open('ridge_test.pickle', 'wb'))

In [34]:
lr_pred = model_lr.predict(X_valid)
ridge_pred = model_ridge.predict(X_valid)
enet_pred = model_enet.predict(X_valid)
gb_pred = model_gb.predict(X_valid)

In [35]:
lr_test = model_lr.predict(X_test)
ridge_test = model_ridge.predict(X_test)
enet_test = model_enet.predict(X_test)
gb_test = model_gb.predict(X_test)

In [37]:
test_1st = pd.DataFrame(ridge_pred, columns = ['ridge'])

test_1st['enet'] = enet_pred
test_1st['gb'] = gb_pred

In [ ]:
test_ = pd.DataFrame(r, columns = ['ridge'])
test_['ridge'] = ridge_test
test_['enet'] = enet_test
test_['gb'] = gb_test

In [ ]:
test_model = LinearRegression(n_jobs=-1)

In [ ]:
test_1st.drop('label',axis=1,inplace=True)
test_model.fit(test_1st,Y_valid)

In [ ]:
ensemble_pred = test_model.predict(test_1st).clip(0,20)

In [ ]:
rmse = sqrt(mean_squared_error(ensemble_pred, Y_valid))
rmse